# 영화진흥위원회 오픈API

In [1]:
import requests
import pandas as pd
import time
import math

/Users/swoo64/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. API 전체 목록 불러오기

In [4]:
# 여기에 발급받은 API 키를 입력하세요.
API_KEY = "0fedb3690ed2dbeb688fb9f77f9b7fdc"

# 영화 목록 API의 기본 URL
BASE_URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"

In [7]:
# 전체 영화 데이터를 저장할 리스트 초기화
all_movies_list = []

# 페이지 처리를 위한 변수 초기화
cur_page = 204
item_per_page = 100 # 한 페이지에 표시할 항목 수 (최대값으로 설정 권장)

print("영화 목록 데이터 추출을 시작합니다...")

while True:
    # API 요청에 필요한 파라미터 설정
    params = {
        'key': API_KEY,
        'curPage': str(cur_page),
        'itemPerPage': str(item_per_page),
    }

    try:
        # API 요청 보내기
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # 오류가 발생하면 예외를 발생시킴

        # JSON 응답 파싱
        data = response.json()
        movie_list = data.get('movieListResult', {}).get('movieList', [])

        # 더 이상 가져올 영화 목록이 없으면 반복 종료
        if not movie_list:
            print("더 이상 가져올 데이터가 없어 추출을 종료합니다.")
            break

        # 현재 페이지의 영화 목록을 전체 리스트에 추가
        all_movies_list.extend(movie_list)

        print(f"페이지 {cur_page} 처리 완료 | 수집된 영화 수: {len(all_movies_list)}개")

        # 다음 페이지로 이동
        cur_page += 1

        # 서버 부하를 줄이기 위한 짧은 대기 시간
        time.sleep(0.5)

    except requests.exceptions.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        break
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")
        break

print(f"\n총 {len(all_movies_list)}개의 영화 데이터를 성공적으로 수집했습니다.")

영화 목록 데이터 추출을 시작합니다...
페이지 204 처리 완료 | 수집된 영화 수: 100개
더 이상 가져올 데이터가 없어 추출을 종료합니다.

총 100개의 영화 데이터를 성공적으로 수집했습니다.


In [ ]:
movie_data=pd.DataFrame(all_movies_list)

| 응답 필드 | 값 | 설명 |
|---|---|---|
| movieCd | 문자열 | 영화코드를 출력합니다. |
| movieNm | 문자열 | 영화명(국문)을 출력합니다. |
| movieNmEn | 문자열 | 영화명(영문)을 출력합니다. |
| prdtYear | 문자열 | 제작연도를 출력합니다. |
| openDt | 문자열 | 개봉일을 출력합니다. |
| typeNm | 문자열 | 영화유형을 출력합니다. |
| prdtStatNm | 문자열 | 제작상태를 출력합니다. |
| nationAlt | 문자열 | 제작국가(전체)를 출력합니다. |
| genreAlt | 문자열 | 영화장르(전체)를 출력합니다. |
| repNationNm | 문자열 | 대표 제작국가명을 출력합니다. |
| repGenreNm | 문자열 | 대표 장르명을 출력합니다. |
| directors | 문자열 | 영화감독을 나타냅니다. |
| peopleNm | 문자열 | 영화감독명을 출력합니다. |
| companys | 문자열 | 제작사를 나타냅니다. |
| companyCd | 문자열 | 제작사 코드를 출력합니다. |
| companyNm | 문자열 | 제작사명을 출력합니다. |

In [ ]:
movie_data.head()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companys
0,20172126,(100ft),(100ft),2016,,단편,기타,"한국,미국",기타,한국,기타,[{'peopleNm': '김민정'}],[]
1,20256927,포풍추영,The Shadow’s Edge,2025,20250920,장편,개봉예정,"홍콩,중국","액션,범죄",홍콩,액션,[{'peopleNm': '래리 양'}],[]
2,20192905,근본주의자,A Fundamentalist,2019,,장편,기타,한국,"코미디,드라마,가족,액션",한국,코미디,[{'peopleNm': '고봉수'}],"[{'companyCd': '20189602', 'companyNm': '고브로 필..."
3,20256401,우리 우린,,,,기타,개봉예정,한국,드라마,한국,드라마,[],[]
4,2025A015,혈전,To Live through Death,2024,,장편,기타,중국,액션,중국,액션,[],[]


## 2. 전체 데이터 저장 (전처리X)

In [ ]:
movie_data.to_csv("movie_full_list.csv", index=False, encoding='utf-8-sig')